# MNIST with CNN using MXNet

Code mostly from http://mxnet.io/tutorials/python/mnist.html

**NOTES**  
Whereas Keras and Tensorflow use has y as one-hot incoded, MXNet takes the unique integers e.g. [5, 0, 1 ...]

## Setup

In [2]:
#import time
import mxnet as mx # for some reason, loading mxnet before the dataset from tensorflow crashes the kernel
#import logging
#logging.getLogger().setLevel(logging.DEBUG)

mnist = mx.test_utils.get_mnist()

In [3]:
#X_train = mnist.train.images
#X_test = mnist.test.images
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')/255

#y_train = mnist.train.labels
#y_test = mnist.test.labels

## Define network

In [4]:
data = mx.symbol.Variable('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=32)
relu1 = mx.sym.Activation(data=conv1, act_type="relu")
pool1 = mx.sym.Pooling(data=relu1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=64)
relu2 = mx.sym.Activation(data=conv2, act_type="relu")
pool2 = mx.sym.Pooling(data=relu2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.Flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=1024)
relu3 = mx.sym.Activation(data=fc1, act_type="relu")
# dropout
drop = mx.sym.Dropout(data=relu3, p=0.5)
# second fullc
fc2 = mx.sym.FullyConnected(data=drop, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

## Train

In [5]:
mod = mx.mod.Module(lenet, context=mx.gpu(0))

t0 = time.time()
mod.fit(mx.io.NDArrayIter(X_train, label=y_train, batch_size=50, shuffle=True),
        num_epoch=10,
        optimizer='adadelta',
        batch_end_callback = mx.callback.Speedometer(50, 200)
       )

print "training time = {}".format(time.time() - t0)

INFO:root:Epoch[0] Batch [200]	Speed: 6286.26 samples/sec	Train-accuracy=0.449055
INFO:root:Epoch[0] Batch [400]	Speed: 6984.10 samples/sec	Train-accuracy=0.943400
INFO:root:Epoch[0] Batch [600]	Speed: 7060.91 samples/sec	Train-accuracy=0.963500
INFO:root:Epoch[0] Batch [800]	Speed: 6555.76 samples/sec	Train-accuracy=0.968500
INFO:root:Epoch[0] Batch [1000]	Speed: 6950.40 samples/sec	Train-accuracy=0.973800
INFO:root:Epoch[0] Train-accuracy=0.975678
INFO:root:Epoch[0] Time cost=8.899
INFO:root:Epoch[1] Batch [200]	Speed: 6519.75 samples/sec	Train-accuracy=0.979403
INFO:root:Epoch[1] Batch [400]	Speed: 6558.97 samples/sec	Train-accuracy=0.980500
INFO:root:Epoch[1] Batch [600]	Speed: 6835.64 samples/sec	Train-accuracy=0.983300
INFO:root:Epoch[1] Batch [800]	Speed: 7413.14 samples/sec	Train-accuracy=0.983700
INFO:root:Epoch[1] Batch [1000]	Speed: 7622.35 samples/sec	Train-accuracy=0.985100
INFO:root:Epoch[1] Train-accuracy=0.987136
INFO:root:Epoch[1] Time cost=8.695
INFO:root:Epoch[2] Bat

training time = 91.1322901249


## Evaluate

In [6]:
metric = mx.metric.Accuracy()
mod.score(mx.io.NDArrayIter(X_test, label=y_test, batch_size=100), metric)

[('accuracy', 0.9918)]

## Store model

In [20]:
lenet.save('./cfr_cnn_mnist_symbol.mxnet')
mod.save_params('./cfr_cnn_mnist_module.mxnet')

In [21]:
import boto3
s3 = boto3.resource('s3')
s3.Bucket('jakechenawspublic').upload_file('./cfr_cnn_mnist_symbol.mxnet',
                                           'tutorials/mlstack_demo/models/cfr_cnn_mnist_symbol.mxnet')
s3.Bucket('jakechenawspublic').upload_file('./cfr_cnn_mnist_module.mxnet',
                                           'tutorials/mlstack_demo/models/cfr_cnn_mnist_module.mxnet')

DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7fcb6760a398>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function _handler at 0x7fcb676099b0>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7fcb67686aa0>
DEBUG:botocore.args:The s3 config key is not a dictionary type, ignoring its value of: None
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Defaulting to S3 virtual host style addressing with path style addressing fallback.
DEBUG:boto3.resources.factory:Loading s3:s3
DEBUG:boto3.resources.factory:Loading s3:Bucket
DEBUG:boto3.resources.model:Renaming Bucket attribute name
DEBUG:botocore.hooks:Event creating-resource-class.s3.Bucket: calling handler <function _handler at 0x7fcb67609aa0>
DEBUG:s3transfer.utils:Acquiring 0
DEBUG:s3transfer.tasks

## Predict function

In [45]:
import boto3
import mxnet as mx
import numpy as np
from mxnet.io import NDArrayIter

s3 = boto3.resource('s3')
bucket = s3.Bucket('jakechenawspublic')
bucket.download_file('tutorials/mlstack_demo/models/cfr_cnn_mnist_symbol.mxnet', './symbol.mxnet')
bucket.download_file('tutorials/mlstack_demo/models/cfr_cnn_mnist_module.mxnet', './module.mxnet')
bucket.download_file('temp.csv', './temp.csv')

DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7fcb6760a398>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function _handler at 0x7fcb676099b0>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7fcb67686aa0>
DEBUG:botocore.args:The s3 config key is not a dictionary type, ignoring its value of: None
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Defaulting to S3 virtual host style addressing with path style addressing fallback.
DEBUG:boto3.resources.factory:Loading s3:s3
DEBUG:boto3.resources.factory:Loading s3:Bucket
DEBUG:boto3.resources.model:Renaming Bucket attribute name
DEBUG:botocore.hooks:Event creating-resource-class.s3.Bucket: calling handler <function _handler at 0x7fcb67609aa0>
DEBUG:s3transfer.utils:Acquiring 0
DEBUG:s3transfer.tasks

In [47]:
f = open('./temp.csv')
image = f.readlines()[0]
image = np.array(image.split(',')[1:])
image = image.reshape(1,1,28,28)

cnn = mx.symbol.load('./symbol.mxnet')
model = mx.mod.Module(cnn, context=mx.gpu(0))
model.bind(NDArrayIter(image).provide_data, for_training=False)
model.load_params('./module.mxnet')

y_pred = model.predict(NDArrayIter(image))

In [ ]:
## WIP ##
import boto3
s3 = boto3.resource('s3')
s3.Object('jakechenawspublic').put(Body=y_pred)